## Notebook used to generate a fake supplier dataset and create the recommandation module. 

### Dataset recommandation fournisseur

In [157]:
import pandas as pd
import numpy as np
import random
import time
import math
import geopy.distance

#### Créer les colonnes du datset

In [158]:
df = pd.DataFrame([])

Répartition des fermes: 25 fermes d'élevage, 100 de fruits et légumes, 75 de céréals

In [159]:
from random import randint

noms_fermes = pd.DataFrame(np.zeros(200))

#fermes d'élevge
noms_fermes.iloc[:10] = ['Les produits laitiers de Dédé', 'Beaux bauvins', 'New Ranch', 'Oh la vache', 'La Rave des Bêtes', 'Le toro', 'Les Cochonets', 'La chèvre de Monsieur', 'Les poules cools', 'La main à la pâte']

nom = ['Le Cochon', 'Le Troupeau', 'Le Poulailler', 'Le Poulet', 'Le Bauvin', 'Le coq', 'Le toro', 'Le champs', 'Le lait', 'Escargot']
adjectif = ['Vert', 'Blanc', 'Bavard', 'Fêtard', 'Détendu', 'Abrité', 'Ensoleillé', 'Heureux', 'Joli', 'Bleu', 'de Jean-Mi', 'du coin', 'régional', 'national', 'de qualité', 'de Bretagne', 'de Normandie', 'en folie', 'à la ferme', 'qui chante', 'qui danse']

def inventeur_de_nom(noms_fermes, nom, adjectif, nb_noms, debut):
    for i in range(nb_noms):
        n = randint(0, len(nom)-1)
        m = randint(0, len(adjectif)-1)
        noms_fermes.iloc[i+debut,0] = nom[n] + ' ' + adjectif[m]
    return noms_fermes

noms_fermes = inventeur_de_nom(noms_fermes, nom, adjectif, 40, 10)


In [160]:
noms_fermes.iloc[20]

0    Le lait qui chante
Name: 20, dtype: object

In [161]:
#fermes de fruits et légumes
noms_fermes.iloc[50:60, 0] = ['La belle ferme', 'Les produits de mamie', 'Fruits et légumes biologiques et écologiques', 
                            'La Maison verte', 'La New Récolte', 'Le Cœur vert', 'La saison de la moisson', 'Soleil et pluie', 
                            'Les cueillettes', 'Dans les bois']

In [162]:
nom = ['La Paille', 'La rosée', 'La main', 'La tournée', 'La région', 'Perspective', 'Recherche', 'La cabane', 'La maison', 'La variété', 'La collection']
adjectif = ['Verte', 'Bio', 'Bio', 'de saison', 'Fraîche', 'Pour la vie', 'depuis toujours', 'certifiée', 'honnête', 'sublime']

noms_fermes = inventeur_de_nom(noms_fermes, nom, adjectif, 80, 60)

In [163]:
import math

def definisseur_de_nom(noms_fermes, nom, adjectif, nb_noms, debut):
    for i in range(3):
        for j in range(19):
            noms_fermes.iloc[i*19+j+debut, 0] = nom[i] + ' ' + adjectif[j]
        noms_fermes.iloc[3*19+i+debut, 0] = nom[i] + ' ' + adjectif[i]
    
    return noms_fermes

In [164]:
noms_fermes[20:]

,0
20,Le lait qui chante
21,Le coq du coin
22,Le coq Bavard
23,Le toro à la ferme
24,Le Bauvin Bleu
...,...
195,0.0
196,0.0
197,0.0
198,0.0


In [165]:
#fermes de céréals
nom = ['La Ferme de', 'Exploitation de', 'La culture de', 'Le blé de']
proprio = ['Pauline', 'M. Alain', 'Jean Pierre', 'M. Bec', 'Merie Brenon', 'Mme Paul Gauthier', 'M. Yves Laurent', 
        'Saint Romain', 'M. Becquart', 'M. Dubec', 'Jeanne', 'Thomas Berger', 'Paul Artisan', 
        'Mme Line Maizeray', 'Avondance', 'Louise Puisenval', 'M. Jean Pres', 'M. Christian', 'M. Pierre Mas', 'Mme Saulcy']

noms_fermes = definisseur_de_nom(noms_fermes, nom, proprio, 60, 140)

In [166]:
noms_fermes

,0
0,Les produits laitiers de Dédé
1,Beaux bauvins
2,New Ranch
3,Oh la vache
4,La Rave des Bêtes
...,...
195,La culture de M. Christian
196,La culture de M. Pierre Mas
197,La Ferme de Pauline
198,Exploitation de M. Alain


In [167]:
df['Nom de la ferme'] = noms_fermes

In [168]:
df['Nom de la ferme']

0      Les produits laitiers de Dédé
1                      Beaux bauvins
2                          New Ranch
3                        Oh la vache
4                  La Rave des Bêtes
                   ...              
195       La culture de M. Christian
196      La culture de M. Pierre Mas
197              La Ferme de Pauline
198         Exploitation de M. Alain
199        La culture de Jean Pierre
Name: Nom de la ferme, Length: 200, dtype: object

In [169]:
df['Produit issu de la ferme'] = np.zeros((200,))
df['Produit issu de la ferme'][:60] = ['parmesan']*60
df['Produit issu de la ferme'][60:140] = ['fraise']*80
df['Produit issu de la ferme'][140:] = ['champignon']*60

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


##### Ville et localisation de la ferme

In [170]:
villes = pd.read_csv("fr.csv")
villes= villes.drop_duplicates(subset ="city", inplace = False)

In [171]:
villes.capital.value_counts(dropna=False)

NaN        9189
minor       303
admin        12
primary       1
Name: capital, dtype: int64

In [172]:
# Localistaion de la ferme

temp = villes[~villes.capital.isin(["minor","primary","admin"])]
villes_fermes = random.sample(list(temp.city),200)
df['city'] = villes_fermes
temp = villes[["city","lat","lng"]]
df = pd.merge(df,temp, on='city')

##### Surface

In [173]:
df['Surface'] = np.random.choice(range(10, 150),len(df),replace=True)

##### Emmission du mode de transport utilisé (en g CO2 / km)

In [174]:
# random entre 400 et 600 g CO2 / km
df['Emmission transport'] = np.random.randint(400, 600, size=(200,))

##### Emmision du packaging de transport utilisé (en g CO2 / kg)

In [175]:
df_packaging = pd.read_csv('data/emballages.csv')

In [176]:
df_packaging.head()

,Catégorie,Matière,kg équivalent CO2 / tonne,Taux de recyclage,Id matière,Recyclé,Type emballage
0,Papier,Pâte à papier primaire,297.0,62.0,1,0,Pochette
1,Papier,Pâte à papier primaire recyclée,317.0,52.0,2,1,Pochette
2,Papier,Pâte à papier Eucalyptus,440.0,61.5,3,0,Pochette
3,Papier,Pâte à papier Résineux,410.0,56.5,4,0,Pochette
4,Papier,Pâte à papier Feuillus,400.0,60.5,5,0,Pochette


In [177]:
df_packaging['Catégorie'].unique()

array(['Papier', 'Carton', 'Plastique', 'Polystyrène', 'PVC', 'Nylon',
       'Verre', 'Bois'], dtype=object)

In [178]:
df['Type packaging'] = np.zeros((200,))
df['Emmission type packaging'] = np.zeros((200,))
df['Masse packaging'] = np.zeros((200,))

In [179]:
# faire quelques recherches sur range de random
sub_mat = df_packaging[df_packaging.Catégorie.isin(['Plastique', 'Carton', 'Polystyrène', 'Plastique'])]

for i in range(200):
    type = np.random.randint(0, 10)
    df['Type packaging'][i] = sub_mat.iloc[type,1]
    df['Emmission type packaging'][i] = sub_mat.iloc[type,2]
    df['Masse packaging'][i] = np.random.random()*0.01 + 0.2
    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipyk

In [180]:
df

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging
0,Les produits laitiers de Dédé,parmesan,Maubec,43.8439,5.1394,128,527,Carton recyclé,670.0,0.204995
1,Beaux bauvins,parmesan,Sainte-Jamme-sur-Sarthe,48.1428,0.1683,48,534,Plastique transparent,2350.0,0.206382
2,New Ranch,parmesan,Vendin-lès-Béthune,50.5422,2.6064,36,576,Carton primaire,390.0,0.204540
3,Oh la vache,parmesan,Pommerit-le-Vicomte,48.6192,-3.0883,72,448,Carton primaire,390.0,0.204832
4,La Rave des Bêtes,parmesan,Plérin,48.5344,-2.7708,82,545,Carton primaire,390.0,0.203431
...,...,...,...,...,...,...,...,...,...,...
195,La culture de M. Christian,champignon,Étain,49.2128,5.6364,30,564,Carton recyclé,670.0,0.205464
196,La culture de M. Pierre Mas,champignon,Saint-Léger-du-Bourg-Denis,49.4344,1.1506,53,493,Plastique blanc,2025.0,0.202017
197,La Ferme de Pauline,champignon,Saint-Jean-de-Thouars,46.9642,-0.2117,137,599,Carton recyclé,670.0,0.203256
198,Exploitation de M. Alain,champignon,Rontalon,45.6603,4.6319,105,527,Plastique blanc,2025.0,0.201343


##### Label bas carbone, bio, produit fermier et AOP (Appelation Origine Protégée)

In [181]:
# random de 0 (non) et 1 (oui)
df['Label bas carbone'] = np.random.randint(0, 2, size=(200,))

In [182]:
# random de 0 (non) et 1 (oui)
df['Label bio'] = np.random.randint(0, 2, size=(200,))

In [183]:
# random de 0 (non) et 1 (oui)
df['Label produit fermier'] = np.random.randint(0, 2, size=(200,))

In [184]:
# random de 0 (non) et 1 (oui)
df['Label aop'] = np.random.randint(0, 2, size=(200,))


In [185]:
df.head()

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging,Label bas carbone,Label bio,Label produit fermier,Label aop
0,Les produits laitiers de Dédé,parmesan,Maubec,43.8439,5.1394,128,527,Carton recyclé,670.0,0.204995,1,0,1,0
1,Beaux bauvins,parmesan,Sainte-Jamme-sur-Sarthe,48.1428,0.1683,48,534,Plastique transparent,2350.0,0.206382,0,1,1,0
2,New Ranch,parmesan,Vendin-lès-Béthune,50.5422,2.6064,36,576,Carton primaire,390.0,0.204540,1,1,1,0
3,Oh la vache,parmesan,Pommerit-le-Vicomte,48.6192,-3.0883,72,448,Carton primaire,390.0,0.204832,1,0,0,0
4,La Rave des Bêtes,parmesan,Plérin,48.5344,-2.7708,82,545,Carton primaire,390.0,0.203431,0,1,1,0


In [186]:
villes.iloc[3,0]

'Marseille'

### --> Upload data sur BQ

### Example

In [187]:
ville_acheteur = 'Lyon'
nom_ferme = 'New Ranch'
produit = 'champignon'

##### Distance parcourue jusque production (en km)

In [188]:
# calcul de la distance 
def calcul_distance(data, villes, ville_acheteur):
    temp = villes[villes.city == ville_acheteur].reset_index(drop=True)
    data['Distance'] = list(zip(data.lat, data.lng))
    data['Distance'] = data['Distance'].apply(lambda x : geopy.distance.geodesic(x, (temp['lat'][0],temp['lng'][0])).km)
    return df


#### Créer le score métric du dataset 

In [189]:
# random.random retourne chiffre entre 0 et 1
df['score'] = np.random.random(size=(200,))+0.5

In [190]:
x_0 = df[df['Nom de la ferme'] == nom_ferme]

In [191]:
df.columns

Index(['Nom de la ferme', 'Produit issu de la ferme', 'city', 'lat', 'lng',
       'Surface', 'Emmission transport', 'Type packaging',
       'Emmission type packaging', 'Masse packaging', 'Label bas carbone',
       'Label bio', 'Label produit fermier', 'Label aop', 'score'],
      dtype='object')

In [192]:
def calcul_score(df, villes, ville_acheteur, produit, nom_ferme):
    
    df = calcul_distance(df, villes, ville_acheteur)
    df['Ville acheteur'] = [ville_acheteur]*200
    
    #définir la ferme et le produit par rapport  auquel on calcule le score
    x_0 = df[df['Nom de la ferme'] == nom_ferme].reset_index(drop=True)
    
    #choisir fermes qui produisent le meme produit uniquement
    sub_df = df[(df['Produit issu de la ferme'] == produit) 
                & (df['Label bio'] == x_0['Label bio'][0])]

    num_columns = ['Surface', 'Emmission transport', 'Emmission type packaging', 'Masse packaging', 'Distance']
    cat_columns = ['Label bas carbone', 'Label bio', 'Label produit fermier', 'Label aop']
    
    delta_num = pd.DataFrame([])
    for c in num_columns:
        #mesurer les deltas en pourcentage pour les colonnes numériques
        delta_num[c] = sub_df[c].apply(lambda x: (x - x_0[c])/x_0[c])

    delta_cat = pd.DataFrame([])
    for c in cat_columns:
        #mesurer les deltas net pour les colonnes catégoriques (0 et 1 des labels)
        delta_cat[c] = sub_df[c].apply(lambda x: (x - x_0[c]))
    
    sub_df['CO2'] = sub_df['Distance'].multiply(sub_df['Emmission transport'])+sub_df['Emmission type packaging'].multiply(sub_df['Masse packaging'])
    
    x_0['CO2'] = x_0['Distance']*x_0['Emmission transport']+x_0['Emmission type packaging']*x_0['Masse packaging']
    
    sub_df['% CO2 reco vs init'] = 1 + sub_df['CO2'].apply(lambda x: (x - x_0['CO2'])/x_0['CO2'])
    sub_df['CO2 norm'] = sub_df['CO2'].apply(lambda x: (x- min(sub_df['CO2']))/(max(sub_df['CO2'])-min(sub_df['CO2'])))                                                                                                                         
    sub_df['score'] = 0.7 - ((sub_df['CO2 norm']) - 0.2 * sub_df['Label bas carbone'] - 0.1 * sub_df['Label aop'])
    
    sub_df = sub_df.sort_values('score', ascending=True)
    
    return sub_df


In [193]:
sub_df = calcul_score(df, villes, ville_acheteur, produit, nom_ferme)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [194]:
sub_df.head()

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging,Label bas carbone,Label bio,Label produit fermier,Label aop,score,Distance,Ville acheteur,CO2,% CO2 reco vs init,CO2 norm
190,La culture de Paul Artisan,champignon,Saint-Nicolas-du-Pélem,48.3131,-3.1644,55,507,Carton primaire,390.0,0.208617,0,1,1,0,-0.300000,670.876627,Lyon,340215.810485,1.060068,1.000000
157,La Ferme de M. Christian,champignon,Ploumoguer,48.4036,-4.7228,148,420,Carton primaire,390.0,0.209342,0,1,1,0,-0.263335,782.726966,Lyon,328826.969221,1.024582,0.963335
149,La Ferme de M. Dubec,champignon,Haveluy,50.3514,3.4028,66,568,Carton primaire,390.0,0.203387,0,1,1,0,-0.158802,521.616223,Lyon,296357.335760,0.923411,0.858802
154,La Ferme de Avondance,champignon,Arthon-en-Retz,47.1164,-1.9386,96,487,Carton recyclé,670.0,0.208401,0,1,1,1,0.044896,542.109784,Lyon,264147.093828,0.823048,0.755104
173,Exploitation de Avondance,champignon,Villereau,50.2461,3.6967,137,581,Carton recyclé,670.0,0.208875,1,1,1,0,0.048313,506.036747,Lyon,294147.296188,0.916525,0.851687


In [201]:
def calcul_score_sans_ferme(data, villes, ville_acheteur, ingredient):
    temp = data.copy()
    
    #temp['Ville acheteur'] = [ville_acheteur]*200
    
    temp = temp[temp['Produit issu de la ferme'] == ingredient].reset_index(drop=True)
    
    temp['CO2'] = temp['Distance'].multiply(temp['Emmission transport'])+temp['Emmission type packaging'].multiply(temp['Masse packaging'])
    
    temp['CO2 norm'] = temp['CO2'].apply(lambda x: (x- min(temp['CO2']))/(max(temp['CO2'])-min(temp['CO2'])))                                                                                                                         
    temp['score'] = 0.7 - ((temp['CO2 norm']) - 0.2 * temp['Label bas carbone'] - 0.1 * temp['Label aop'])
    
    reco = temp.sort_values('score', ascending = False)
    
    return reco

In [209]:
df = calcul_distance(df, villes, 'Nantes')

In [210]:
reco = calcul_score_sans_ferme(df, villes, 'Nantes', 'fraise')
reco.iloc[:10,:]

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging,Label bas carbone,Label bio,Label produit fermier,Label aop,score,Distance,Ville acheteur,CO2,CO2 norm
2,La région Pour la vie,fraise,Saligny,46.8078,-1.4253,63,541,Carton primaire,390.0,0.202876,1,0,1,1,1.000000,46.632377,Lyon,25307.237880,0.000000
47,La collection Bio,fraise,Chalonnes-sur-Loire,47.3506,-0.7639,15,442,Carton recyclé,670.0,0.208307,1,1,0,1,0.994799,61.472298,Lyon,27310.321472,0.005201
51,La collection de saison,fraise,Mauléon,46.9228,-0.7497,142,480,Plastique transparent,2350.0,0.207258,1,0,0,0,0.878110,69.272744,Lyon,33737.972514,0.021890
54,Perspective depuis toujours,fraise,Vern-sur-Seiche,48.0447,-1.6003,37,448,Carton recyclé,670.0,0.202542,1,1,1,0,0.858372,91.973291,Lyon,41339.737733,0.041628
4,La cabane Verte,fraise,Plouray,48.1458,-3.3881,48,589,Carton primaire,390.0,0.207234,1,1,0,1,0.802273,172.118916,Lyon,101458.862367,0.197727
29,La rosée Verte,fraise,Chevaigné,48.2119,-1.6297,15,582,Plastique blanc,2025.0,0.200021,1,0,0,0,0.797455,110.645973,Lyon,64800.997947,0.102545
72,La maison depuis toujours,fraise,Châteaubernard,45.6725,-0.3144,141,553,Carton recyclé,670.0,0.205100,1,1,0,1,0.783359,196.394511,Lyon,108743.581205,0.216641
74,La cabane certifiée,fraise,La Possonnière,47.3753,-0.6864,134,566,Carton recyclé,670.0,0.204322,0,0,1,1,0.765686,67.819394,Lyon,38522.673004,0.034314
8,La variété Bio,fraise,Bourgneuf,46.1669,-1.0242,14,405,Plastique blanc,2025.0,0.209192,0,1,0,1,0.734579,123.653060,Lyon,50503.104142,0.065421
59,La maison honnête,fraise,Parcé-sur-Sarthe,47.8436,-0.2014,92,413,Carton recyclé,670.0,0.201439,0,0,1,1,0.733192,123.250296,Lyon,51037.336021,0.066808


In [67]:
def recommender_fournisseur(data, villes, ville_acheteur, produit, nom_ferme = None, metric = 'score'):
    
    if nom_ferme is None : 
        df_resultat = calcul_score_sans_ferme(data, villes, ville_acheteur, produit).sort_values(by = metric, ascending = False)
        resultat = df_resultat.head(10)
    else : 
        df_resultat = calcul_score(data, villes, ville_acheteur, produit).sort_values(by = metric, ascending = False)
        resultat = df_resultat.head(10)
    
    return resultat

In [72]:
recommender_fournisseur(df, villes, 'Paris', 'fraise')

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging,Label bas carbone,Label bio,Label produit fermier,Label aop,score,Distance,Ville acheteur,CO2,CO2 norm
50,La région Pour la vie,fraise,Ternay,45.6022,4.8106,35,447,Carton primaire,390.0,0.207403,1,1,0,1,1.000000,17.687624,Lyon,7987.255142,0.000000
23,La main Bio,fraise,Yenne,45.7042,5.7589,30,424,Carton recyclé,670.0,0.204002,1,0,0,1,0.937587,71.791757,Lyon,30576.386319,0.062413
57,Recherche Bio,fraise,Saint-Jean-de-Moirans,45.3411,5.5822,86,597,Carton recyclé,670.0,0.209324,1,0,0,1,0.899057,74.340684,Lyon,44521.634835,0.100943
54,La cabane Bio,fraise,Dingy-Saint-Clair,45.9117,6.2222,102,541,polystyrène primaire,2824.0,0.201171,1,0,1,1,0.858018,108.700161,Lyon,59374.893595,0.141982
43,La tournée Verte,fraise,Martignat,46.2094,5.6089,120,492,Polystyrène recyclé,2820.0,0.208089,1,1,0,0,0.814760,77.746745,Lyon,38838.209797,0.085240
37,La rosée sublime,fraise,Craponne-sur-Arzon,45.3308,3.8481,106,559,Carton recyclé,670.0,0.205678,1,0,0,0,0.781183,90.971333,Lyon,50990.779417,0.118817
4,La collection de saison,fraise,Saint-Paul-en-Chablais,46.3786,6.6303,103,591,polystyrène thermoformé,3600.0,0.206143,1,1,0,1,0.767518,154.631932,Lyon,92129.585704,0.232482
36,La variété Bio,fraise,Marchaux,47.3233,6.1328,13,518,Carton primaire,390.0,0.205331,1,0,0,1,0.735499,200.074185,Lyon,103718.506539,0.264501
39,La Paille de saison,fraise,Jonquerettes,43.9472,4.9331,125,528,styrène,2700.0,0.206698,1,1,1,1,0.726440,201.588834,Lyon,106996.987805,0.273560
45,La maison sublime,fraise,Meythet,45.9153,6.0928,148,419,Carton recyclé,670.0,0.201844,0,1,0,1,0.707261,98.847144,Lyon,41552.188730,0.092739
